In [1]:
import os
os.chdir("..")
print(os.getcwd())

import sys
from pathlib import Path
from importlib import reload

/hs/fs08/data/group-brueggen/tmartinez/diffusion/src


In [31]:
'a' * 0

''

In [45]:
import datasets.images as dsimg
reload(dsimg)
import datasets.cutouts as dscut
reload(dscut)
import cProfile
import pandas as pd
from utils.paths import LOFAR_RES_CAT, PLAYGROUND_DIR

cutout_file = Path(
    '/hs/fs08/data/group-brueggen/tmartinez/diffusion/image_data/LOFAR/cutouts/cutouts_80p_optC_.hdf5'
)
def test_run():
    catalog = pd.read_csv(LOFAR_RES_CAT)
    # Select 10k random entries as subset of the catalog
    catalog = catalog.sample(1000, random_state=42)
    dscut.get_cutouts(catalog=catalog, fname_comment="profile")

cProfile.run("test_run()", str(PLAYGROUND_DIR / 'restats'))
# Test clipping with 0-clip
# dsimg.clip_cutouts(cutout_file, 0)

Looping through mosaics...: 100%|██████████| 560/560 [00:42<00:00, 13.12it/s]


Updating catalog...
Saving images...
Saving catalog... (this might take a while)
Done.


In [55]:
import pstats
from pstats import SortKey
p = pstats.Stats(str(PLAYGROUND_DIR / 'restats'))
p.sort_stats('ncalls', 'cumtime').print_stats(.5)

Thu Apr 18 14:28:46 2024    /hs/fs08/data/group-brueggen/tmartinez/analysis_results/playground/restats

         21530325 function calls (21058215 primitive calls) in 47.218 seconds

   Ordered by: call count, cumulative time
   List reduced from 1651 to 826 due to restriction <0.5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
3554265/3452558    0.845    0.000    1.605    0.000 {built-in method builtins.isinstance}
1671453/1395834    0.310    0.000    0.424    0.000 {built-in method builtins.len}
1010480/994480    0.203    0.000    0.535    0.000 {built-in method builtins.getattr}
   745996    0.337    0.000    0.701    0.000 /hs/fs08/data/group-brueggen/tmartinez/envs/cenv_diffusion/lib/python3.11/site-packages/pandas/core/dtypes/generic.py:42(_instancecheck)
   745996    0.263    0.000    0.350    0.000 /hs/fs08/data/group-brueggen/tmartinez/envs/cenv_diffusion/lib/python3.11/site-packages/pandas/core/dtypes/generic.py:37(_check)
   521360    0.124    0.00

In [61]:
p.print_callers('/nddata/utils.py:547')

   Ordered by: call count, cumulative time
   List reduced from 1651 to 1 due to restriction <'/nddata/utils.py:547'>

Function                                                                                                                       was called by...
                                                                                                                                   ncalls  tottime  cumtime
/hs/fs08/data/group-brueggen/tmartinez/envs/cenv_diffusion/lib/python3.11/site-packages/astropy/nddata/utils.py:547(__init__)  <-    1000    0.064    1.998  /hs/fs08/data/group-brueggen/tmartinez/diffusion/src/datasets/cutouts.py:16(single_cutout)




In [10]:
import h5py
import numpy as np
import pandas as pd

with h5py.File(cutout_file, "r") as f:
    images = np.array(f["cutouts"])
catalog = pd.read_hdf(cutout_file, key="catalog")

In [15]:
h5py.check_vlen_dtype(None) is None

True

In [29]:
import pandas as pd
import numpy as np

def mixed_type_cols(catalog):
    # Assume catalog is your DataFrame
    mixed_type_columns = catalog.apply(lambda x: x.map(type).nunique() > 1)

    # Get the names of the columns with mixed data types
    mixed_type_column_names = mixed_type_columns[mixed_type_columns].index.tolist()

    # For each column with mixed data types, print the name, unique data types, counts, percentages, and an example
    for column_name in mixed_type_column_names:
        print(f"Column: {column_name}")
        type_counts = catalog[column_name].map(lambda x: str(type(x))).value_counts()
        type_percentages = type_counts / len(catalog) * 100
        for dtype, count in type_counts.items():
            percentage = type_percentages[dtype]
            # Get an example of the current type
            example = catalog[column_name][catalog[column_name].map(lambda x: str(type(x))) == dtype].iloc[0]
            print(f"  Type: {dtype}, Count: {count}, Percentage: {percentage:.2f}%, Example: {example}")

In [28]:
import utils.paths as paths

dr2_cat = pd.read_csv(
    paths.LOFAR_RES_CAT
)

In [30]:
mixed_type_cols(dr2_cat)

Column: Position_from
  Type: <class 'str'>, Count: 314915, Percentage: 99.98%, Example: LR
  Type: <class 'float'>, Count: 54, Percentage: 0.02%, Example: nan
Column: Renamed_from
  Type: <class 'float'>, Count: 277902, Percentage: 88.23%, Example: nan
  Type: <class 'str'>, Count: 37067, Percentage: 11.77%, Example: ILTJ000000.57+321232.6
Column: UID_L
  Type: <class 'str'>, Count: 202997, Percentage: 64.45%, Example: 3598p335_0001491
  Type: <class 'float'>, Count: 111972, Percentage: 35.55%, Example: nan
Column: UNWISE_OBJID
  Type: <class 'str'>, Count: 228664, Percentage: 72.60%, Example: 0000p333o0018472
  Type: <class 'float'>, Count: 86305, Percentage: 27.40%, Example: nan
Column: ID_NAME
  Type: <class 'str'>, Count: 238315, Percentage: 75.66%, Example: 0000p333o0018472
  Type: <class 'float'>, Count: 76654, Percentage: 24.34%, Example: nan
Column: z_source
  Type: <class 'str'>, Count: 165987, Percentage: 52.70%, Example: Phot
  Type: <class 'float'>, Count: 148982, Percenta

In [41]:
s = dr2_cat.loc[42]
print(type(s['RA']))
s.to_frame().loc['RA'].dtype

<class 'numpy.float64'>


dtype('O')